In [1]:
import numpy as np

In [2]:
def gaussian(x, mu, sigma):
    """
    Computes the probability of a given data point x, given the mean and covariance of a Gaussian distribution.

    Parameters:
        - x : np.ndarray, data point
        - mu : np.ndarray, mean
        - sigma : np.ndarray, covariance
    
    Returns:
        - p : float, probability of x given the mean and covariance
    """
    d = len(x)
    p = 1 / (np.sqrt((2 * np.pi) ** d * np.linalg.det(sigma))) * np.exp(-0.5 * (x - mu).T @ np.linalg.inv(sigma) @ (x - mu))
    return p

def likelihood(X, mu, sigma, pi):
    """
    Computes the likelihood of the data given the parameters of the GMM.

    Parameters:
        - X : np.ndarray, data
        - mu : np.ndarray, means
        - sigma : np.ndarray, covariances
        - pi : np.ndarray, mixing coefficients
    
    Returns:
        - likelihood : float, likelihood of the data given the parameters
    """
    N = X.shape[0]
    K = len(mu)
    L = 0
    for i in range(N):
        p = 0
        for k in range(K):
            p += pi[k] * gaussian(X[i], mu[k], sigma[k])
        L += np.log(p)
    return L


def EM_algorithm(mu, sigma, pi, X, tol=1e-6, max_iter=1000):
    """
    Performs the EM algorithm for a Gaussian Mixture Model, Returning updated parameters after likelihood is maximised.

    Parameters:
        - mu : np.ndarray, means
        - sigma : np.ndarray, covariances
        - pi : np.ndarray, mixing coefficients
        - X : np.ndarray, data
    
    Returns:
        - mu : np.ndarray, means
        - sigma : np.ndarray, covariances
        - pi : np.ndarray, mixing coefficients




    """
    K = len(mu)
    N = X.shape[0]
    d = X.shape[1]
    L = likelihood(X, mu, sigma, pi)
    for i in range(max_iter):
        
        # Epectation-step
        
        gamma = np.zeros((N, K))
        
        for i in range(N):
            for k in range(K):
                gamma[i, k] = pi[k] * gaussian(X[i], mu[k], sigma[k])
            gamma[i] /= np.sum(gamma[i])
        
        # Maximise Likelihood  step

        Nk = np.sum(gamma, axis=0)
        mu_new = np.zeros((K, d))
        sigma_new = np.zeros((K, d, d))
        pi_new = np.zeros(K)

        for k in range(K):
            mu_new[k] = np.sum(gamma[:, k].reshape(-1, 1) * X, axis=0) / Nk[k]
            sigma_new[k] = np.sum(gamma[i, k] * np.outer(X[i] - mu_new[k], X[i] - mu_new[k]) for i in range(N)) / Nk[k]
            pi_new[k] = Nk[k] / N

        mu = mu_new
        sigma = sigma_new
        pi = pi_new

        L_new = likelihood(X, mu, sigma, pi)

        if np.abs(L_new - L) < tol:
            break
        
        if i == max_iter - 1:
            print("Max iterations reached without convergence.")

        L = L_new

    
    return mu, sigma, pi
